M76 Homework 1 (due 6/28/2024) \\
Author/Collaboration: Warren Shepard and Sri Korandla \\
Credit: Used Chat-GPT ONLY to redo the tic-tac-toe interface


In [ ]:
# import statements
import random
import random
from IPython.display import display, clear_output
import ipywidgets as widgets

In the below code, the computer plays "O" and the user plays "X" no matter who goes first.

In [ ]:
# returns list of tuples of coords of empty positions
def get_empty_positions(board):
    positions = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == " ":
                positions.append((i, j))
    return positions

# returns true if board is emtpy
def is_board_empty(board):
    return " " not in [row for row in board]

def check_winner(board, mark):
    win_conditions = [
        [board[0][0], board[0][1], board[0][2]],
        [board[1][0], board[1][1], board[1][2]],
        [board[2][0], board[2][1], board[2][2]],
        [board[0][0], board[1][0], board[2][0]],
        [board[0][1], board[1][1], board[2][1]],
        [board[0][2], board[1][2], board[2][2]],
        [board[0][0], board[1][1], board[2][2]],
        [board[2][0], board[1][1], board[0][2]],
    ]
    return [mark, mark, mark] in win_conditions

def update_board():
    clear_output(wait=True)
    display(board_ui, status, reset_button, first_move_toggle)
    for i in range(3):
        for j in range(3):
            buttons[i][j].description = board[i][j]
            buttons[i][j].disabled = board[i][j] != " "

def disable_buttons():
    for row in buttons:
        for button in row:
            button.disabled = True

def reset_game(b):
    global board, user_first
    board = [[" " for _ in range(3)] for _ in range(3)]
    status.value = ""
    for row in buttons:
        for button in row:
            button.description = " "
            button.disabled = False
    user_first = first_move_toggle.value == "User"
    update_board()
    if not user_first:
        computer_move()

def user_move(row, col):
    if board[row][col] == " ":
        board[row][col] = "X"
        update_board()
        if check_winner(board, "X"):
            status.value = "Congratulations! You win!"
            disable_buttons()
            return
        if not get_empty_positions(board):
            status.value = "It's a draw!"
            disable_buttons()
            return
        computer_move()

def computer_move():
    best_score, best_move = minimax(board)
    board[best_move[0]][best_move[1]] = "O"
    update_board()
    if check_winner(board, "O"):
        status.value = "Computer wins! Better luck next time."
        disable_buttons()
        return
    if not get_empty_positions(board):
        status.value = "It's a draw!"
        disable_buttons()
        return

In [ ]:
# Initialize global variables
board = [[" " for _ in range(3)] for _ in range(3)]
user_first = False

# Create the board buttons
buttons = [[widgets.Button(description=" ", layout=widgets.Layout(width="50px", height="50px")) for _ in range(3)] for _ in range(3)]

# Attach the button click events
for i in range(3):
    for j in range(3):
        buttons[i][j].on_click(lambda b, row=i, col=j: user_move(row, col))

# Display the board
board_ui = widgets.VBox([widgets.HBox(row) for row in buttons])
status = widgets.Label(value="")
reset_button = widgets.Button(description="Reset Game")
reset_button.on_click(reset_game)
first_move_toggle = widgets.ToggleButtons(
    options=['User', 'Computer'],
    description='First move:',
    disabled=False,
    button_style=''
)
first_move_toggle.value = 'Computer'  # Set default to Computer

display(first_move_toggle, board_ui, status, reset_button)
update_board()

# Make the computer move first if the default is set to Computer
if first_move_toggle.value == 'Computer':
    computer_move()

Label(value='')

Button(description='Reset Game', style=ButtonStyle())

ToggleButtons(description='First move:', options=('User', 'Computer'), value='User')

In [ ]:
# Initialize memoization table
memo = {}

# returns coordinate of optimal next move
def minimax(board):
    # base case: check if there exists a winner or tie, or if if the solution is already stored
    if str(board) in memo:
        return memo[str(board)]
    if check_winner(board, "O"):
        return (1, None)
    elif check_winner(board, "X"):
        return (-1, None)
    elif not get_empty_positions(board):
        return (0, None)

    # recursive case: calculate score for each empty position
    empty_positions = get_empty_positions(board)
    best_score = -1
    best_move = empty_positions[0]

    # iterate over empty positions
    for pos in empty_positions:
        new_board = [row[:] for row in board]
        new_board[pos[0]][pos[1]] = "O"

        # check if the computer wins
        if check_winner(board, "O"):
          return (1, pos)

        min_score = 1

        # iterate over remaining empty positions
        for player_pos in get_empty_positions(new_board):
            new_board[player_pos[0]][player_pos[1]] = "X"
            score = minimax(new_board)[0]  # recursive call to minimax

            # undo move and calculate min score
            new_board[player_pos[0]][player_pos[1]] = " "
            min_score = min(min_score, score)

        # update best score and move
        if min_score > best_score:
            best_score = min_score
            best_move = pos

        new_board[pos[0]][pos[1]] = " "  # Undo move

    # update memo table and return
    memo[str(board)] = (best_score, best_move)
    return (best_score, best_move)

Given the current board, we can calculate the best next move for the computer recursively. Suppose that the "score" is 1 if the computer wins, -1 if the user wins, and 0 if it's a tie; in other words, the computer is trying to maximize the score while the user is trying to minimize it.

The recurrence equality is given as follows. Let $S$ be the current empty positions. and let $\Phi$ be the "score function" (computed recursivly), taking in the computers next move and the players next move as parameter. Then, the optimal next move $s^*$ for the bot is
  \begin{align}
          s^* = \text{MAX}_{s \in S} \, \text{MIN}_{s' \in S} \, \Phi(s, s')
          \end{align}

This is indeed a simpflication of a famous algorithm called Minimax

##Proof of Correctness for the minimax Function in Tic Tac Toe
To prove the correctness of the minimax function, we will use mathematical induction to  demonstrate that the function adheres to the principles of the minimax algorithm, ensuring it correctly identifies the optimal move for player "O" (The Computer), to ensure a victory / tie if "O" plays first. We are trying to show that this function evaluates the optimal move for the board configuration with $\textit{n}$ $\geq$ 0 empty positions.

###Base Case

We first show that the base cases correctly handle terminal states when $\textit{n}$ = 0

* If player "O" wins, the function returns 1.
* If player "X" wins, the function returns -1.
* If there are no empty positions (indicating a tie, as the board has not returned that either plays has hit a win state), the function returns 0.

These conditions correctly identify the terminal states and return the appropriate values:

* 1 for a win by "O".
* -1 for a win by "X".
* 0 for a tie.


### Inductive Step

For non-terminal states, we need to show that the function correctly evaluates the optimal move for "O" using the minimax principle.

### Inductive Hypothesis

Assume that the minimax function correctly evaluates the optimal move for any board configuration with $\textit{n}$ $\geq$ 0 empty positions.
\subsubsection*{Inductive Step}

We must show that the minimax function correctly evaluates the optimal move for any board configuration with $\textit{n}$ + 1 empty positions.

Consider a board with $\textit{n}$ + 1 empty positions. The function does the following:

1. $\textit{Retrieves the empty positions on the board.}$
2. $\textit{Initializes the best score to a value less than any possible game score and selects the first empty position as the initial best move.}$
3. $\textit{Iterates over each empty position, simulating placing "O" on the board.}$
4. $\textit{For each move by Player "O", initializes the minimum score for Player "X" to the maximum possible value.}$
5. $\textit{Simulates "X"'s move by iterating over remaining empty positions (excluding the move just made by "O"), recursively calling minimax to evaluate the score, and updating the min score for "X".}$
6. $\textit{Updates the best score and move if the current move's score is better than the best score.}$

By the inductive hypothesis, the recursive call to minimax correctly evaluates the board with $\textit{n}$ empty positions. Therefore, for each move by Player "O", the function correctly considers the optimal counter-move by "X" and determines the minimum score that "X" can force. The outer loop of of iterating over each empty position and placing an "O" ensures that Player "O" chooses the move that maximizes its minimum guaranteed score (follows the MinMax strategy).

By induction, the minimax function correctly evaluates the optimal move for any board configuration with $\textit{n}$ +1 empty positions. Combined with the base case, this proves that the minimax function is correct for determining the optimal move for player "O" in Tic Tac Toe. Thus, the minimax function will be correct and terminate properly.

### Impletation notes
We chose to use a memoization table to trade off space or increased speed. This makes the first move (by the bot) about three seconds faster.